<a href="https://colab.research.google.com/github/kamalakbari7/PySpark/blob/main/Challenge2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge

## Download and install Spark

In [1]:
!ls

sample_data


In [5]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,046 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,342 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,169 

## Setup environment

**[Jan 2023 update]**
- Google colab recently upgraded to Python 3.8. Unfortunately this breaks spark 2.3.1. 
- Please use the code below where we install from the pyspark package instead

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=940a4c845ab841625f4efc1caaef1e078cac10a9e627e202d4ca50ea93dc434c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [7]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!ls -l

--2023-05-17 18:13:39--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [<=>                 ]   1.71G  2.88MB/s    in 10m 40s 

2023-05-17 18:24:20 (2.74 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1840824738]

total 1839920
-rw-r--r--  1 root root 1840824738 May 17 11:01 'rows.csv?accessType=DOWNLOAD'
drwxr-xr-x  1 root root       4096 May 16 19:33  sample_data
drwxrwxr-x 11 1000 1000       4096 Jun  1  2018  spark-2.3.1-bin-hadoop2.7
-rw-r--r--  1 root root   43233280 May 17 18:13  spark-2.3.1-bin-hadoop2.7.tgz


In [8]:
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
!ls -l

total 1839920
-rw-r--r--  1 root root 1840824738 May 17 11:01 reported-crimes.csv
drwxr-xr-x  1 root root       4096 May 16 19:33 sample_data
drwxrwxr-x 11 1000 1000       4096 Jun  1  2018 spark-2.3.1-bin-hadoop2.7
-rw-r--r--  1 root root   43233280 May 17 18:13 spark-2.3.1-bin-hadoop2.7.tgz


In [ ]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

## Challenge questions

**What is the most frequently reported non-criminal activity?**

In [ ]:
rc.chache()
rc.count()

In [ ]:
rc.show(5)

In [ ]:
rc.select(col('Primary Type')).distinct().count()

In [ ]:
rc.select(col('Primary Type')).distinct().orderBy(col('Primary Type')).show(35, truncate=False)

In [ ]:
nc = rc.filter(('Primary Type'== 'NON - CRIMINAL')|('Primary Type'== 'NON-CRIMINAL')|('Primary Type'== 'NON - CRIMINAL (SUBJECT SPECIFIED)'))
nc.show(50)

In [ ]:
nc.groupBy(col('Description')).count().orderBy('count',ascending=False).show(truncate=False)

**Which day of the week has the most number of reported crime?** and **Using a bar chart, plot which day of the week has the most number of reported crime.**

In [ ]:
from pyspark.sql.functions import dayofweek

In [ ]:
help(dayofweek)

In [ ]:
rc.show()

In [ ]:
rc.select(col('Date'),dayofweek(col('Date'))).show(5)

In [ ]:
from pyspark.sql.functions import date_format

In [ ]:
help(date_format)

In [ ]:
rc.select(col('Date'),dayofweek(col('Date')),date_format(col('Date'),'E')).show(5)

In [ ]:
rc.groupBy(date_format(col('Date'),'E')).count().orderBy('count',ascending=False).show()

In [ ]:
rc.groupBy(date_format(col('Date'),'E')).count().collect()

In [ ]:
dow = [x[0] for x in rc.groupBy(date_format(col('Date'),'E')).count().collect()]
dow

In [ ]:
cnt = [x[1] for x in rc.groupBy(date_format(col('Date'),'E')).count().collect()]
cnt

In [ ]:
import pandas as pd
import matplotlib as plt

In [ ]:
cp = pd.Dataframe({'Day_of_week': dow, 'Count': cnt})
cp.head()

In [ ]:
cp.sort_values('Count', ascending=False).plot(kind = 'bar',colour = 'olive', x = 'Day_of_week',y = 'Count')
plt.xlabel('Day of the week')
plt.ylabel('No. of reported crimes')
plt.title('No. of reported crimes sper day of the week')